## Project: Perception Pick & Place
### Writeup Template: You can use this file as a template for your writeup if you want to submit it as a markdown file, but feel free to use some other method and submit a pdf if you prefer.

---


# Required Steps for a Passing Submission:
1. Extract features and train an SVM model on new objects (see `pick_list_*.yaml` in `/pr2_robot/config/` for the list of models you'll be trying to identify). 
2. Write a ROS node and subscribe to `/pr2/world/points` topic. This topic contains noisy point cloud data that you must work with.
3. Use filtering and RANSAC plane fitting to isolate the objects of interest from the rest of the scene.
4. Apply Euclidean clustering to create separate clusters for individual items.
5. Perform object recognition on these objects and assign them labels (markers in RViz).
6. Calculate the centroid (average in x, y and z) of the set of points belonging to that each object.
7. Create ROS messages containing the details of each object (name, pick_pose, etc.) and write these messages out to `.yaml` files, one for each of the 3 scenarios (`test1-3.world` in `/pr2_robot/worlds/`).  [See the example `output.yaml` for details on what the output should look like.](https://github.com/udacity/RoboND-Perception-Project/blob/master/pr2_robot/config/output.yaml)  
8. Submit a link to your GitHub repo for the project or the Python code for your perception pipeline and your output `.yaml` files (3 `.yaml` files, one for each test world).  You must have correctly identified 100% of objects from `pick_list_1.yaml` for `test1.world`, 80% of items from `pick_list_2.yaml` for `test2.world` and 75% of items from `pick_list_3.yaml` in `test3.world`.
9. Congratulations!  Your Done!

# Extra Challenges: Complete the Pick & Place
7. To create a collision map, publish a point cloud to the `/pr2/3d_map/points` topic and make sure you change the `point_cloud_topic` to `/pr2/3d_map/points` in `sensors.yaml` in the `/pr2_robot/config/` directory. This topic is read by Moveit!, which uses this point cloud input to generate a collision map, allowing the robot to plan its trajectory.  Keep in mind that later when you go to pick up an object, you must first remove it from this point cloud so it is removed from the collision map!
8. Rotate the robot to generate collision map of table sides. This can be accomplished by publishing joint angle value(in radians) to `/pr2/world_joint_controller/command`
9. Rotate the robot back to its original state.
10. Create a ROS Client for the “pick_place_routine” rosservice.  In the required steps above, you already created the messages you need to use this service. Checkout the [PickPlace.srv](https://github.com/udacity/RoboND-Perception-Project/tree/master/pr2_robot/srv) file to find out what arguments you must pass to this service.
11. If everything was done correctly, when you pass the appropriate messages to the `pick_place_routine` service, the selected arm will perform pick and place operation and display trajectory in the RViz window
12. Place all the objects from your pick list in their respective dropoff box and you have completed the challenge!
13. Looking for a bigger challenge?  Load up the `challenge.world` scenario and see if you can get your perception pipeline working there!

## [Rubric](https://review.udacity.com/#!/rubrics/1067/view) Points
### Here I will consider the rubric points individually and describe how I addressed each point in my implementation.  

---
### Writeup / README

#### 1. Provide a Writeup / README that includes all the rubric points and how you addressed each one.  You can submit your writeup as markdown or pdf.  

You're reading it!

### Exercise 1, 2 and 3 pipeline implemented
#### 1. Complete Exercise 1 steps. Pipeline for filtering and RANSAC plane fitting implemented.

After doing a few iteration steps in the implementation of the VoxelGrid SownSampling filter I selected the LEAF_SIZE = 0.01

This reduces the computation time more than 3 times and we still have a good representation of the objects in the cloud points

|LEAF_SIZE| POINTS | TIME(ms) |
|:---:|:---:|:---:|
|1|23|790|
|1|868|114|
|1|55336|480|
|0.0001|202627|1530|

For the PassTrought Filter the values for only select the table and the objects on it where: (max, min) = (1.1 , 0.7)

Finally we implemented the RANSAC algorithm for the extraction of the table points an its counterpart. A distance threshold of 0.01 did the job for our points cloud

The following collection of images ilustrate the implementation process for each step:

- Top-left : the original data
- bottom-left: VoxelGrid + PassTrought
- Top-right: RANSAC (objects part)
- Bottom-left: RANSAC (table part)

![](./images/exercise1.png)


#### 2. Complete Exercise 2 steps: Pipeline including clustering for segmentation implemented.  

The good documentation of the lessons makes easy follow the steps.

After make the ros publisher and introduce the code from the exercise_1 we can visualize the objects without the table in the working environment.

![](./images/exercise2_objects.png)

So the interesting part of this projects is iterate over the parameters of the Euclidean Clustering to isolate correctly the objects on the table. The paramenters that I selected after working a little bit where:

- ClusterTolerance = 0.05
- MinClusterSize = 20
- MaxClusterSize = 2000

However, even when this parameters correctly clustered the objects the edge of the table was clustered as an object. To avoid this I changed the axis_min of the passThrough filter to **0.76**. The result is shown in the following image.

![](./images/exercise2_color_cluster.png)

#### 2. Complete Exercise 3 Steps.  Features extracted and SVM trained.  Object recognition implemented.

In this exercise we build all parts together to finally be able to do an object recognition. Is was very interesting implement the histograms part (colors and normal surfaces )and see how this tool can be used to extract features for an SVM training.

Tweaking a little bit wiht the parameters we can achieve a good performance ~80% in my case. However much more higher performances should be achieved training the model a little bit more. With the initial parameters provide by Udacity the confusion matrix had an overall recognition of 65%. Modifiing the number of posses and playing with parameters as the number of bins in the histogras or the kernel in the SVM algorithm we can improve this solutions as shown in the table.

|RGB_HSV|bins|orientations|kernel|Accuracy Score|
|:---:|:---:|:---:|:---:|:---:|
|HSV|32|10|linear| 0.657|
|HSV|32|10|rbf| 0.685|
|HSV|32|10|linear|0.757|
|HSV|64|20|linear|0.771|
|HSV|64|20|rbf|0.800|

And the normalized confusion matrix for the final training set

![](./images/exercise3-NormconfusionMatrix6Run.png)

After the training write the ros node to publish the results is easy, in the next image we can see that the new code can recognize various objects on the table. As expected, the model is not very accurate and give  so It will need more training or differents parameters in the next steps.

![](./images/object_recog.png)

### Pick and Place Setup

#### 1. For all three tabletop setups (`test*.world`), perform object recognition, then read in respective pick list (`pick_list_*.yaml`). Next construct the messages that would comprise a valid `PickPlace` request output them to `.yaml` format.


Spend some time at the end to discuss your code, what techniques you used, what worked and why, where the implementation might fail and how you might improve it if you were going to pursue this project further.  



